In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
import gseapy as gp

In [ ]:
homer_path = ''
# homer_path = 'export PATH=$PATH:/sc/projects/sci-kretzmer/chair/software/tools/homer/bin/;'
env = ''
# env = 'eval "$(conda shell.bash hook)";conda activate /sc/projects/sci-kretzmer/chair/software/conda/metilene3;export MPLBACKEND=qtagg;'

In [ ]:
os.system(env+'python ../metilene3/metilene3.py \
    -i ./data/GSE121721_glioma.input.tsv \
    -o ./GBM \
    -t 16 \
    -n 4 \
    -plot True -anno hg19\
')

In [ ]:
dmrmean_m_rename = pd.read_table('./GBM/heatmap.tsv', index_col=0)
sinfo = dmrmean_m_rename[[]]
sinfo['group'] = [i.split(' ')[0] for i in dmrmean_m_rename.index]
sinfo['subtype'] = [i.split('=')[1].split('_')[0] for i in dmrmean_m_rename.index]
sinfo.index = [i.split('_')[-1] for i in sinfo.index]
sinfo

In [ ]:
dmrs = pd.read_table('./GBM/DMRs.tsv')
dmrs

RNA

In [ ]:
bed = pd.read_table('./data/geo/GSE121720_RAW/gencode.v19.annotation.onlygenes.bed', header=None)
bed.index = bed[0]+'.'+bed[1].astype(str)+'.'+bed[2].astype(str)

raw = pd.read_table('./data/geo/raw.genes.npz')
raw.index = raw["#'chr'"]+'.'+raw["'start'"].astype(str)+'.'+raw["'end'"].astype(str)

raw = pd.DataFrame(raw.drop(columns=["#'chr'","'start'","'end'"]))
raw.columns = [i.split('_')[1] for i in raw.columns]
raw.index = raw.index.map(bed[3].to_dict())
raw.index.name = 'ensgene'
raw = raw.loc[~raw.index.duplicated()]

normal = {
    'frontal':'NBr1',
    'occipital':'NBr2',
    'parietal':'NBr3',
    'temporal':'NBr4'
}
newidx = list(raw.columns)
for i in range(len(newidx)):
    if newidx[i] in normal.keys():
        newidx[i] = normal[newidx[i]]
raw.columns = newidx
raw = raw.loc[raw.index[(raw.T>1).mean()>0.5]]
raw

In [ ]:
tpm = pd.read_table('./data/geo/GSE121720_RNAseq_expression_matrix_TPMs.txt.gz').T
tpm = (tpm+0.1).applymap(np.log10)
normal = {
    'frontal':'NBr1',
    'occipital':'NBr2',
    'parietal':'NBr3',
    'temporal':'NBr4'
}
newidx = list(tpm.index)
for i in range(len(newidx)):
    if newidx[i] in normal.keys():
        newidx[i] = normal[newidx[i]]
tpm.index = newidx
tpm = tpm.loc[raw.columns]
tpm

In [ ]:
def rundeseq(Y0,Y1):
    i = ''.join(Y1)+'vs'+''.join(Y0)
    selected = sinfo.loc[sinfo['group'].isin(Y0+Y1)]
    selected['y'] = 1*(selected['group'].isin(Y0))
    
    raw[selected.index].astype(int).to_csv(\
        './GBM/deseq2_exp_'+i+'.csv')
    selected[['y']].to_csv(\
        './GBM/deseq2_y_'+i+'.csv')
    os.system('Rscript ./deseq2.r \
            ./GBM/deseq2_exp_'+i+'.csv \
            ./GBM/deseq2_y_'+i+'.csv \
            ./GBM/deseq2_out_'+i+'.csv')

In [ ]:
gene_names = {}
with open('./data/geo/Homo_sapiens.GRCh38.111.gtf', 'r') as gtf_file:
    for line in gtf_file:
        if line.startswith('#'):
            continue
        fields = line.strip().split('\t')
        if len(fields) < 9:
            continue
        attributes = fields[8]
        attr_dict = {}
        if attributes.find('gene_id')>-1 and attributes.find('gene_name')>-1 :
            gene_names[attributes.split('gene_id "')[1].split('"')[0]] = \
                attributes.split('gene_name "')[1].split('"')[0]

In [ ]:
Y1 = ['G2']
Y0 = ['G1']


rundeseq(Y0,Y1)

selected = sinfo.loc[sinfo['group'].isin(Y0+Y1)]
selected['y'] = 1*(selected['group'].isin(Y1))

i = ''.join(Y1)+'vs'+''.join(Y0)

deseq2 = pd.read_csv('./GBM/deseq2_out_'+i+'.csv', index_col=0)
deseq2['ENSG'] = deseq2.index 
deseq2.index = [j.split('.')[0] for j in deseq2.index]
deseq2.index = deseq2.index.map(gene_names)
deseq2 = deseq2.sort_values('pvalue')
deseq2 = deseq2.loc[(~deseq2.index.isna())&(~deseq2.index.duplicated())].dropna()

degsea = gp.prerank(rnk=deseq2['log2FoldChange'],
                     gene_sets='./data/geo/h.all.v2023.2.Hs.symbols.gmt',
                     threads=10, outdir=None, seed=1, 
                    )

degsea.res2d['absNES'] = degsea.res2d['NES'].apply(abs)
degsea.res2d.sort_values('absNES', ascending=False).head(10)

In [ ]:
i = 'P0|2|3|0'
gs = sorted(set(dmrs.loc[(dmrs['DMTree'].str.contains(i.replace('|','\|')))&\
                            (dmrs['meandiffabs']>0.1)]['SYMBOL'].dropna()))
print(i,len(gs))
dmgsea = gp.enrichr(gene_list=gs,
                 gene_sets='./data/geo/h.all.v2023.2.Hs.symbols.gmt',
                 outdir=None,
                )

In [ ]:
dgsea = pd.merge(dmgsea.res2d, degsea.res2d, on='Term', how='outer').dropna()

dgsea['combine_pvalues'] = dgsea.apply(lambda x:st.combine_pvalues([x['P-value'],x['NOM p-val']]).pvalue,axis=1)
dgsea['combine_pvalues'] += dgsea.loc[dgsea['combine_pvalues']>0]['combine_pvalues'].min()*0.1

plt.subplots(figsize=[8,4])
ax = sns.scatterplot(x=dgsea['Odds Ratio'], y=dgsea['NES'],\
                     s=30-30*dgsea['combine_pvalues'].apply(np.log10), color='gray')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

print(st.spearmanr(dgsea['Odds Ratio'], dgsea['NES'], nan_policy='omit'))
dgsea.loc[dgsea['combine_pvalues']<0.01].sort_values(['NES'])